In [2]:
import pandas as pd
from pandasql import sqldf



In [3]:
# Load the cleaned dataset
file_path = 'LA Clippers Data Challenge F&B Dataset cleaned.csv'
df = pd.read_csv(file_path)



In [4]:

# Setup pandasql
pysqldf = lambda q: sqldf(q, globals())

df.head()


,OrderID,OrderStatus,CustomerID,ArrivalDatetime,DepartureDatetime,GroupSize,ProductID,ProductName,ProductQuantity,ProductFullPrice,PaymentStatus,OrderTotalQuantity,EmptyOrder,Category,SubCategory
0,1159,paid,1006,2023-03-23 00:13:00,2023-03-23 00:15:00,1,44532400000000,Aquafina- 20 oz bottle,1,7.15,success,2,No,Non-alcohol,Water
1,1159,paid,1006,2023-03-23 00:13:00,2023-03-23 00:15:00,1,44532400000000,Pepperoni Pizza,1,18.15,success,2,No,Food,Hot food
2,1160,paid,1001,2023-03-23 00:15:00,2023-03-23 00:17:00,2,44532400000000,Starry- 20 oz bottle,1,7.15,success,4,No,Non-alcohol,Non-alcohol Beverage
3,1160,paid,1001,2023-03-23 00:15:00,2023-03-23 00:17:00,2,44532400000000,Pink's Hot Dog,2,18.70,success,4,No,Food,Hot food
4,1160,paid,1001,2023-03-23 00:15:00,2023-03-23 00:17:00,2,44532400000000,Sour Patch Candy,1,6.05,success,4,No,Food,Snack


In [5]:


# Display the first few rows and column names for inspection
print(df.head())
print(df.columns)


   OrderID OrderStatus  CustomerID      ArrivalDatetime    DepartureDatetime  \
0     1159        paid        1006  2023-03-23 00:13:00  2023-03-23 00:15:00   
1     1159        paid        1006  2023-03-23 00:13:00  2023-03-23 00:15:00   
2     1160        paid        1001  2023-03-23 00:15:00  2023-03-23 00:17:00   
3     1160        paid        1001  2023-03-23 00:15:00  2023-03-23 00:17:00   
4     1160        paid        1001  2023-03-23 00:15:00  2023-03-23 00:17:00   

   GroupSize       ProductID             ProductName  ProductQuantity  \
0          1  44532400000000  Aquafina- 20 oz bottle                1   
1          1  44532400000000         Pepperoni Pizza                1   
2          2  44532400000000    Starry- 20 oz bottle                1   
3          2  44532400000000          Pink's Hot Dog                2   
4          2  44532400000000        Sour Patch Candy                1   

   ProductFullPrice PaymentStatus  OrderTotalQuantity EmptyOrder     Category  \

In [6]:
# Display the first few rows and column names for inspection
print(df.head())
print(df.columns)


   OrderID OrderStatus  CustomerID      ArrivalDatetime    DepartureDatetime  \
0     1159        paid        1006  2023-03-23 00:13:00  2023-03-23 00:15:00   
1     1159        paid        1006  2023-03-23 00:13:00  2023-03-23 00:15:00   
2     1160        paid        1001  2023-03-23 00:15:00  2023-03-23 00:17:00   
3     1160        paid        1001  2023-03-23 00:15:00  2023-03-23 00:17:00   
4     1160        paid        1001  2023-03-23 00:15:00  2023-03-23 00:17:00   

   GroupSize       ProductID             ProductName  ProductQuantity  \
0          1  44532400000000  Aquafina- 20 oz bottle                1   
1          1  44532400000000         Pepperoni Pizza                1   
2          2  44532400000000    Starry- 20 oz bottle                1   
3          2  44532400000000          Pink's Hot Dog                2   
4          2  44532400000000        Sour Patch Candy                1   

   ProductFullPrice PaymentStatus  OrderTotalQuantity EmptyOrder     Category  \

In [10]:
# Register the DataFrame 'df'
sqldf("SELECT * FROM df", globals())

# Define the SQL query to calculate the average spending for a group of 2 people
query = """
    SELECT AVG(spending) AS avg_spending
    FROM df
    WHERE group_size = 2
"""

# Use pandasql to execute the query
result = sqldf(query, globals())
print(result)


   avg_spending
0          32.5


In [38]:
print(df.columns)

new_data = pd.read_csv('LA Clippers Data Challenge F&B Dataset cleaned.csv')

# Merge the new DataFrame with your existing DataFrame
df = pd.concat([df, new_data], axis=1)

# Print the updated DataFrame to inspect the changes
print(df.columns)


Index(['group_size', 'spending'], dtype='object')
Index(['group_size', 'spending', 'OrderID', 'OrderStatus', 'CustomerID',
       'ArrivalDatetime', 'DepartureDatetime', 'GroupSize', 'ProductID',
       'ProductName', 'ProductQuantity', 'ProductFullPrice', 'PaymentStatus',
       'OrderTotalQuantity', 'EmptyOrder', 'Category', 'SubCategory'],
      dtype='object')


In [39]:
# Setup pandasql
pysqldf = lambda q: sqldf(q, globals())

# Convert ArrivalDatetime to PDT
df['ArrivalDatetime'] = pd.to_datetime(df['ArrivalDatetime'])
df['ArrivalDatetime'] = df['ArrivalDatetime'].dt.tz_localize('UTC').dt.tz_convert('US/Pacific')

# Execute the SQL Query for the highest amount of Food purchased in a day
query_b = """
SELECT DATE(ArrivalDatetime) as Date, MAX(ProductQuantity * ProductFullPrice) as MaxFoodPurchase
FROM df
WHERE Category = 'Food'
GROUP BY Date
ORDER BY MaxFoodPurchase DESC
LIMIT 1;
"""
highest_food_purchase = pysqldf(query_b)
print(highest_food_purchase)


         Date  MaxFoodPurchase
0  2023-05-26            212.5


In [41]:
# Setup pandasql
pysqldf = lambda q: sqldf(q, globals())

query_c ="""
SELECT OrderID,
       COUNT(*) AS NumOfProducts,
       SUM(ProductFullPrice) AS TotalRevenue,
       RANK() OVER (ORDER BY SUM(ProductFullPrice) DESC) AS Rank
FROM df
GROUP BY OrderID
ORDER BY TotalRevenue DESC
LIMIT 5;
"""

result = pysqldf(query_c)
print(result)



   OrderID  NumOfProducts  TotalRevenue  Rank
0    15625              7        187.20     1
1    16197              6        185.36     2
2    18816              9        170.53     3
3    11775             12        157.31     4
4    16094              6        149.06     5
